## Lab 1. Electronic Structure Calculations: Basics

In this lab you will:

* Perform basic *ab initio* calculations in Psi4. 
* Learn how to create an input for evaluation of basic properties. 
* Compute the energy differences between different conformations of the same molecules. 
* Calculate the interaction energies between two molecules. 

Authors: Lyudmila Slipchenko (lslipchenko@purdue.edu; ORCID: 0000-0002-0445-2990) and Victor H. Chavez (gonza445@purdue.edu; ORCID: 0000-0003-3765-2961)

***

In [ ]:
#You will need to import the following modules to run calculations. 
#Scientific Python tools
import numpy as np
import pandas as pd

#Computational Chemistry Tools
import psi4

#Visualization tools
import matplotlib.pyplot as plt
import qcelemental as qc

### Part A: Computing energies of different conformations

In [ ]:
#Let us create a water molecule in psi4. We require a string with three main components:
#Charge and spin multiplicity 
#Geometry (Z-matrix, xyz)
#Symmetry

bent_geometry = psi4.geometry("""
0 1

O
H 1 0.96
H 1 0.96 2 104.5

symmetry c1
""")

#Perform an energy calculation for water using:
e_bent   = psi4.energy("HF/6-31G*", molecule=bent_geometry)

In [ ]:
#The Psi4 geometry class gives the geometrical information to QCElemental for plotting.
print("Bent water molecule")

bent = qc.models.Molecule.from_data(bent_geometry.save_string_xyz())
bent.show()

In [ ]:
#Based on the example of the bent water, define a geometry and make an energy calculation for a linear molecule

#Input as cartesian coordinates for linear water molecule
linear_geometry = psi4.geometry("""
0 1

O 
H 1 0.96
H 1 0.96 2 180.0
""")

#Energy calculation
e_linear   = psi4.energy("HF/6-31G*", molecule=linear_geometry)

In [ ]:
#Confirm that your geometry is correct by showing the structure

print("Linear water molecule")

linear = qc.models.Molecule.from_data(linear_geometry.save_string_xyz())
linear.show()

In [ ]:
#Pandas is a library designed to work nicely with databases. 
#If you followed the naming patterns for the variables, you should see a nice table with your results.

#Print results
water = pd.DataFrame(data = {'Linear':[e_linear], 'Bent':[e_bent]})
water.index=['Energy']
water

### Questions:

a) Compare the total energy of bent and linear water. What are their relative energies? Express the relative energies in terms of kJ/mol, kcal/mol and in hartrees. Keeping in mind that the Boltzmann energy at room temperature $K_{B}T$, is the linear water conformation easily accessible at room temperature?

In [ ]:
#Response

b) **Extra**: You can repeat the same steps with planar and pyramidal ammonia $NH_3$ to see if the "umbrella inversion" process is feasible at room temperature. Is tunnelling a factor in this inversion process?

In [ ]:
#Response

***

### Part B: Computing interaction energies

1. Compute the energies of two water clusters using the following given geometry at **HF/6-31G(d)** level of theory:

    O    -0.089523    0.063946    0.08686  
    H     0.864783    0.058339    0.103755  
    H    -0.329829    0.979459    0.078369  
    O     2.632273   -0.313504   -0.750376  
    H     3.268182   -0.937310   -0.431464  
    H     2.184198   -0.753305   -1.469059
    

2. Show the geometry of the cluster. 
3. Make a table showing energies of two monomers, cluster and the interaction energy.

Hint: Compute the energy of the first monomer $A$ (first three atoms), then compute the energy of the second monomer $B$ (last three atoms), followed by the energy of the dimer $AB$ (all six atoms). 

The total interaction energy: 
$$ E_{int} = E(AB) - (E(A) - E(B)) $$

In [ ]:
#Response
#Each of the monomers in the cluster is a water molecue, we can then recycle the energy of the previous calculation.

#Calculate cluster energy
cluster_geometry = psi4.geometry("""
0 1
     O    -0.089523    0.063946    0.086866
     H     0.864783    0.058339    0.103755
     H    -0.329829    0.979459    0.078369
     O     2.632273   -0.313504   -0.750376
     H     3.268182   -0.937310   -0.431464
     H     2.184198   -0.753305   -1.469059
""")


cluster = qc.models.Molecule.from_data(cluster_geometry.save_string_xyz())
cluster.show()

In [ ]:
e_cluster = psi4.energy('HF/6-31G*', molecule=cluster_geometry)

In [ ]:
interaction = pd.DataFrame(data = {'Monomer':[2*e_bent], 'Cluster':[e_cluster]})
interaction.index=['Energy']
interaction['$E_{int}$ (Hartrees)'] = interaction['Monomer'] - interaction['Cluster']
interaction

### Questions

a) Show the three different energies (monomer, cluster, interaction) in hartrees, kJ/mol and kcal/mol. (Pandas has the ability of multiplying a whole column/row by a constant)

In [ ]:
#Response

***

### Part C: Fun with formaldehyde.

#### 1. With the following geomtry build a formaldehyde molecule:
    
      H   -0.0000000    0.9275885    1.1766889  
      C   -0.0000000    0.0000000    0.6019825  
      H   -0.0000000   -0.9275885    1.1766889  
      O    0.0000000   -0.0000000   -0.6001772  
      
      
#### 2. Perform energy calculation of the molecule at the **HF/6-31G*** level of theory. 

 

In [ ]:
#Define geometry

formal = psi4.geometry("""
noreorient

  H   -0.0000000    0.9275885    1.1766889
  C   -0.0000000    0.0000000    0.6019825
  H   -0.0000000   -0.9275885    1.1766889
  O    0.0000000   -0.0000000   -0.6001772
  
symmetry c1
""")

#Perform calculation to check current energy
energy  = psi4.energy('HF/6-31G*', molecule=formal)
print(energy)

In [ ]:
#It is a good idea to keep the the output file.
#Psi4Numpy prints it to the terminal by default.

psi4.core.set_output_file("optimize.dat")

Let us confirm that the previous geometry for formaldehyde has been optimized.

We use `psi4.optimize` in order to achieve this. 
The syntax for it is the following:

In [ ]:
energy, history = psi4.optimize('HF/6-31G*', molecule=formal, return_history=True)
#return_history has some information about the miminzation.
print(energy)

#### Compare the two energies. What can you notice? 
Now, go the file created "optimize.dat". It includes the SCF calculation of each step in the geometry optimization.  
The last part of the file has the Total Energy per optimization step.  
The extra columns (Delta E, MAX Force, etc) are used as converge parameters.  

In [ ]:
#The optimized geometry can now be extracted with:

history["coordinates"][-1].np

In [ ]:
#Use the updated coordinates for the following parts. 

***

### Orbitals & Density

In this part you will visualize fundamental volumetric information of a molecule, being the electron density and the molecular orbitals. In order to do this, Psi4 needs to know in advance that you are interested in these quantities. The keyword is as follows:

In [ ]:
psi4.set_options({'cubeprop_tasks': ['orbitals', 'density'],
                  'cubeprop_orbitals':  [12],
                  })

In [ ]:
#You can proceed and define the geometry of a formaldehyde molecule. 

In [ ]:
#Response


formal = psi4.geometry("""
0 1
  H   -0.0000000    0.9275885    1.1766889
  C   -0.0000000    0.0000000    0.6019825
  H   -0.0000000   -0.9275885    1.1766889
  O    0.0000000   -0.0000000   -0.6001772
  
symmetry c1
""")

#### The information about the densities and orbitals are contained within Psi4 in a "wavefunction" object. This can be obtained from your usual energy calculation simply by modifying slightly your syntax:

##### e_formal, wfn_formal = psi4.energy('HF/sto-3g', return_wfn=True, molecule = formal)

#### The "wfn_formal" contains much of the information required to perform the calculation, it can give you basic information about the geometry, but also contains important scf quantities like the one particle density matrix. 

In [ ]:
#Use the previous syntax to get the energy and wavefunction object for formaldehyde. 
e_formal, wfn_formal = psi4.energy('HF/sto-3g', return_wfn=True, molecule = formal)

In [ ]:
formal = qc.models.Molecule.from_data(formal.save_string_xyz())
formal.show()

In [ ]:
#Let's visualize the previous quantities, we need the library "blobs". 
#The syntax is the following, you need to use the wavefunction object you just created. 
import blobs
cube = blobs.Cube(wfn_formal)

#By running this cell, blobs should generate a series of files containing the information:
#Density = Alpha density (Da), Beta density (Db), Total density(Dt), Density difference(Ds)
#Orbitals = {Psi_a_n_n-A.cube}_n with n being the number of orbital

In [ ]:
#Plot of Density
cube.plot("Dt.cube")

In [ ]:
#Plots of orbitals
cube.plot("Psi_a_12_12-A.cube", cube_type="orbital")

#### 1. Use the previous syntax to visualize the molecular orbitals and save as an image the 8 occupied and 4 virtual (unoccupied) orbitals. Provide their orbital energies (in Hartrees), and assign the characher (i.e., bonding/antibonding/lone pair, sigma or pi, etc.). The character may be ambiguous, but do your best. Based on this analysis, write down electronic configuration of $CH_2O$. 

**Hint**: Orbital energies can be accessed from the wavefunction as in:
wfn_formal.epsilon_a().np

In [ ]:
#Response

#### 2. Present a geometry in such a way that chemists (not computer), could understand it: write down bbond lenghts and valence angles, not xyz coordinates. Write down the nuclear repulsion and electronic energies in Hartrees (check the output in the terminal)

In [ ]:
#Response

***

### Frequency calculation

#### Perform vibrational frequency calculation of formaldehyde at equilibrium geometry at same level of theory (HF/6-31G*). Instead of using psi4.energy, replace and use psi4.frequencies. It is important that you obtain the wavefunction object just like you did for the cube files. 

In [ ]:
#Tell psi4 that we want to store the information about the normal modes:
psi4.set_options({"NORMAL_MODES_WRITE" : True})

#Frequencies calculation
scf_e, scf_wfn = psi4.frequencies('HF/6-31G*', return_wfn=True)

#### The information about the frequencies can be extracted from the wavefunction. Blobs is also use to examine this results.

In [ ]:
vib = blobs.Freq(scf_wfn)

In [ ]:
#Obtain the frequencies with:
vib.frequencies

In [ ]:
#Visualize the vibrations with:

In [ ]:
vib.plot(vib=6)

#### Inspect vibrational modes of the molecule, practice them over the weekend. 
#### Provide a sketch of each mode, write down its frequency and assignment (symmetric or assymetric stretch, torsion, scissors, etc). 

#### Comparison with to experiment. Compare your finding with availiable experimental data (geometry and frequencies). Are the calculated results accurate? How large are computational errors?

In [ ]:
#Response